In [1]:
# Disable jedi autocompleter
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
import os 
import itertools

### Preprocess Data

In [4]:
def find_unique_items(input_path, output_path):
    """function for finding the unique items in BMS txt file"""
    input_file = open(input_path, "r")
    output_file = open(output_path, "w")

    input_contents = input_file.read()
    item_list = input_contents.split()
    unique_items = set(item_list)
    
    unique_items.difference_update({"-1","-2"})
    
    for item in unique_items:
        output_file.write(str(item) + "\n")

    input_file.close()
    output_file.close()


In [4]:
find_unique_items("./Dataset/BMS1_spmf.txt", "./Dataset/BMS1_item_names.txt")

In [5]:
def sequence_to_binary_matrix(data_path, item_names_path, output_path):
    """
    converts sequences of data into binary matrix format
    and creates a new csv file to store it
    """
    data_file = open(data_path, "r")
    item_names_file = open(item_names_path, "r")
    
    item_names = item_names_file.read().split()    
    
    # create item index dictionary to create binary matrix
    item_index = {}
    idx = 0
    for item in [int(item) for item in item_names]:
        item_index[item] = idx
        idx += 1
    
    # read and clean sequence rows (original form of the data)
    binary_matrix = list()
    with open(data_path) as file:
        for line in file:
            row = [int(value) for value in set(line.rstrip().split()) if value != "-1" and value != "-2"] 
            
            # vector of zeros 
            vec = np.zeros(len(item_names))
            for item in row:
                for key,idx in item_index.items():
                    if item == key:
                        vec[idx] = 1
            binary_matrix.append(vec)    
    
    df = pd.DataFrame(binary_matrix,columns=item_index.keys(), dtype=np.int)
    df.to_csv(output_path, index=None)
    


In [ ]:
sequence_to_binary_matrix("./Dataset/BMS2.txt", "./Dataset/BMS2_item_names.txt", "./Dataset/BMS2_table.csv")

In [3]:
#READ FILE BMS2
file = open("./Dataset/BMS2.txt","r")
read = file.readlines()
lines = list()
for line in read:
    line = line.replace("-2","")
    line = line.replace(" ","")
    line = line[:-2]
    line = line.rstrip("-")
    line= line.replace("-1",",")
    lines.append(line)
file.close()
file_write = open("./Dataset/BMS2_table.csv","w")
for line in lines:
    file_write.write(line+"\n")
file_write.close()

In [ ]:
# BMS2
my_cols = [x for x in range(0,161)]
df = pd.read_csv("./Dataset/BMS2_table.csv",names=my_cols,index_col=None ,engine='c',dtype= 'object')
df = df.where((pd.notnull(df)), None)
df_dummies = pd.get_dummies(df)
df_dummies = df_dummies[:10000]
# sum item with same number after _
columns = list()
for column in df_dummies.columns:
    for column_1 in df_dummies.columns:
        if(column != column_1 and column.split('_')[1] == column_1.split('_')[1] and column.split('_')[1] not in columns ):
            columns.append(column.split('_')[1])

# remove duplicate
columns = list(set(columns))  
zero_data = np.zeros(shape=(len(df_dummies),len(columns)))
df_new = pd.DataFrame(zero_data, columns=columns,dtype='uint8')
# qualcosa non val alcuni sono valori maggiori di 1
# create a transactional data
for column in df_dummies.columns:
    for column_1 in df_dummies.columns:
        if(column != column_1 and column.split('_')[1] == column_1.split('_')[1]):
            if(column.split('_')[0]=='0'):
                colonna_new  = column.split('_')[1]
                if(len(df_new.index[df_new[colonna_new] > 0].tolist())==0):
                    df_new[colonna_new]=df_new[colonna_new] + df_dummies[column] + df_dummies[column_1]
                else:
                    df_new[colonna_new] = df_new[colonna_new] + df_dummies[column_1]
                print(df_new[colonna_new])
                break
                
# ora bisogn aaggiungere len(df) -len(columns) item finti con vlori tutti 0 
# per avere una matrice quadrata (vedi paper)
len(columns)
# write item in a file
with open("./Dataset/BMS2_item_names.txt","w") as f:
    for x in list(df_new.columns):
        f.write(str(x)+"\n")
        
df_new.to_csv("./Dataset/BMS2_10k_records.csv",index=None,header=None)